In [1]:
from quantileRegression import quantileRegression
import numpy as np

Welcome to ROOTaaS 6.06/02


In [2]:
# Variable you want to correct
# ylist = ["R9", "S4", "SigmaIeIe", "EtaWidth", "PhiWidth", "CovarianceIphiIphi", "SigmaRR"]
ylist = ["R9", "SigmaIeIe"]

# Input variables used for the regression
X = ['Pt', 'ScEta', 'Phi', 'rho']  # <<-- list

In [3]:
# Set the quantiles 
quantiles = [ 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9 ]
print "Number of quantiles ", len(quantiles)

Number of quantiles  9


In [4]:
# Initialize the quantile regression object for mc
qr_mc = quantileRegression("mc")

In [5]:
# This is what you want to correct
print "Load the mc dataframe"
startEvtmc = 2000001 
stopEvtmc  = -1
qr_mc.loadDF("/Users/mauro/CMS/Physics/dataMC/nt/double_ele_spring16v2_sync_v1_mc/",
             "cicNoSigmaIetaIeta/trees/",
             ["DYToEE_powheg_13TeV_EBHighR9", "DYToEE_powheg_13TeV_EBLowR9", "DYToEE_powheg_13TeV_EEHighR9", "DYToEE_powheg_13TeV_EELowR9" ],
             startEvtmc, stopEvtmc, 12345)

Load the mc dataframe
NT branches: 
['rho', 'nvtx', 'leadPt', 'leadScEta', 'leadPhi', 'leadR9', 'leadS4', 'leadSigmaIeIe', 'leadEtaWidth', 'leadPhiWidth', 'leadCovarianceIphiIphi', 'leadSigmaRR', 'subleadPt', 'subleadScEta', 'subleadPhi', 'subleadR9', 'subleadS4', 'subleadSigmaIeIe', 'subleadEtaWidth', 'subleadPhiWidth', 'subleadCovarianceIphiIphi', 'subleadSigmaRR']
Reading trees
Number of events  
rho                          2393830
nvtx                         2393830
leadPt                       2393830
leadScEta                    2393830
leadPhi                      2393830
leadR9                       2393830
leadS4                       2393830
leadSigmaIeIe                2393830
leadEtaWidth                 2393830
leadPhiWidth                 2393830
leadCovarianceIphiIphi       2393830
leadSigmaRR                  2393830
subleadPt                    2393830
subleadScEta                 2393830
subleadPhi                   2393830
subleadR9                    2393830
suble

In [6]:
#print "Get the corrected data-mc values"
qr_mc.correctAllY(X, ylist, quantiles )

Loading corrected targets from :  correctedTargets.h5


In [7]:
dfmc = qr_mc.getDF()
# print dfmc

In [8]:
# now get the data
qr_data = quantileRegression("data")
startEvtdata = 0
stopEvtdata   = -1
print "Load the data dataframe"
qr_data.loadDF("/Users/mauro/CMS/Physics/dataMC/nt/double_ele_spring16v2_sync_v1_ichep/",
               "cicNoSigmaIetaIeta/trees/",
               ["Data_13TeV_EBHighR9", "Data_13TeV_EBLowR9", "Data_13TeV_EEHighR9", "Data_13TeV_EELowR9" ],
               startEvtdata, stopEvtdata)

Load the data dataframe
NT branches: 
['rho', 'nvtx', 'leadHLT_Ele27_WPTight_Gsf_vMatch', 'subleadHLT_Ele27_WPTight_Gsf_vMatch', 'leadPt', 'leadScEta', 'leadPhi', 'leadR9', 'leadS4', 'leadSigmaIeIe', 'leadEtaWidth', 'leadPhiWidth', 'leadCovarianceIphiIphi', 'leadSigmaRR', 'subleadPt', 'subleadScEta', 'subleadPhi', 'subleadR9', 'subleadS4', 'subleadSigmaIeIe', 'subleadEtaWidth', 'subleadPhiWidth', 'subleadCovarianceIphiIphi', 'subleadSigmaRR']
Reading trees
Number of events  
rho                                    1037409
nvtx                                   1037409
leadHLT_Ele27_WPTight_Gsf_vMatch       1037409
subleadHLT_Ele27_WPTight_Gsf_vMatch    1037409
leadPt                                 1037409
leadScEta                              1037409
leadPhi                                1037409
leadR9                                 1037409
leadS4                                 1037409
leadSigmaIeIe                          1037409
leadEtaWidth                           1037409
lea

In [9]:
dfdata = qr_data.getDF()
#print dfdata

In [10]:
#plt.scatter(dfdata['ScEta'], dfdata['R9'],marker='.', c='green')
#plt.scatter(dfmc['ScEta'], dfmc['R9'],marker='.', c='red')
#plt.scatter(dfmc['ScEta'], dfmc['R9_corr'],marker='.', c='blue')

In [11]:
#poors man profile plot
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt

def Profile(x,y,nbins,xmin,xmax,ax, color):
    df = DataFrame({'x' : x , 'y' : y})
    
    binedges = xmin + ((xmax-xmin)/nbins) * np.arange(nbins+1)
    df['bin'] = np.digitize(df['x'],binedges)

    # print df
    
    bincenters = xmin + ((xmax-xmin)/nbins)*np.arange(nbins) + ((xmax-xmin)/(2*nbins))
    # print bincenters
    
    ProfileFrame = DataFrame({'bincenters' : bincenters, 'N' : df['bin'].value_counts(sort=False)},index=range(1,nbins+1))
    # ProfileFrame['N'] = ProfileFrame['N'].fillna(1)
    # print ProfileFrame['N']

    bins = ProfileFrame.index.values
    for bin in bins:
        ProfileFrame.ix[bin,'ymean']      = df.ix[df['bin']==bin,'y'].mean()
        ProfileFrame.ix[bin,'yStandDev']  = df.ix[df['bin']==bin,'y'].std()
        ProfileFrame.ix[bin,'yMeanError'] = ProfileFrame.ix[bin,'yStandDev'] / np.sqrt(ProfileFrame.ix[bin,'N'])

    ProfileFrame['ymean']      = ProfileFrame['ymean'].fillna(0)
    ProfileFrame['yMeanError'] = ProfileFrame['yMeanError'].fillna(0)
    #ax.errorbar(ProfileFrame['bincenters'], ProfileFrame['ymean'], yerr=np.array(ProfileFrame['yMeanError']), c=color) #, xerr=(xmax-xmin)/(2*nbins), fmt=None) 
    ax.errorbar(ProfileFrame['bincenters'], ProfileFrame['ymean'], c=color)     
    return ax

In [12]:
# Region definitions

eta = ['EB','EE']
pT  = ['37']
rho = ['12']

mcBinNames = []
dataBinNames = []
binFileNames = []
plotTitles = []

for ieta in eta:
    etabin = ieta
    etacut = ''
    if ieta == 'EB': etacut = '(-1.444 < ScEta & ScEta < 1.444)'
    if ieta == 'EE': etacut = '(ScEta < -1.57 | ScEta > 1.57)'
    # print etabin

    for ipT in range(0,len(pT)+1):
        ptbin = ''
        ptcut = ''
        # print 'ipT', ipT
        if ipT == 0:
            ptbin = 'pTlt'+pT[ipT]
            ptcut = '(Pt <' + pT[ipT] +')'
        elif ipT < len(pT):
            ptbin = 'pT'+pT[ipT-1]+pT[ipT]
            ptcut = '('+pT[ipT-1] +' <= Pt) & ( Pt < '+pT[ipT] +')'
        elif ipT == len(pT):
            ptbin = 'pTgt'+pT[ipT-1]
            ptcut = '( Pt >=' + pT[ipT-1] +')'
        # print "    ", ptbin

        for irho in range(0,len(rho)+1):
            rhobin = ''
            rhocut = ''
            # print 'irho', irho
            if irho == 0:
                rhobin = 'rholt'+rho[irho]
                rhocut = '(rho <' + rho[irho] +')'
            elif irho < len(rho):
                rhobin = 'rho'+rho[irho-1]+rho[irho]
                rhocut = '('+rho[irho-1] +' <= rho) & ( rho < '+rho[irho] +')'
            elif irho == len(rho):
                rhobin = 'rhogt'+rho[irho-1]
                rhocut = '( rho >=' + rho[irho-1] +')'
            # print "        ", rhobin
            
            binFileNames.append('./plots/'+etabin + ptbin + rhobin)
            mcBinNames.append('dfmc.query(\'' + etacut + ' & ' + ptcut + ' & ' + rhocut + '\')')
            dataBinNames.append('dfdata.query(\'' + etacut + ' & ' + ptcut + ' & ' + rhocut + '\')')
            plotTitles.append(etacut + ' & ' + ptcut + ' & ' + rhocut)

#for i in mcBinNames:
#    print i
#for i in dataBinNames:
#    print i
#for i in plotTitles:
#    print i
#for i in binFileNames:
#    print i



In [13]:
# loop over all targets ["R9", "S4", "SigmaIeIe", "EtaWidth", "PhiWidth", "CovarianceIphiIphi", "SigmaRR"]

for yvar in ylist:
    yvarcorr = yvar+"_corr"
    for i in range(0,len(plotTitles)):
        print mcBinNames[i]
        print dataBinNames[i]
        mcbin   = eval(mcBinNames[i])
        databin = eval(dataBinNames[i])

        fig = plt.figure()
        ax = fig.add_subplot(111)
        ax.set_title(plotTitles[i])
        ax.set_xlabel('ScEta')
        ax.set_ylabel(yvar)
        Profile(mcbin['ScEta']  , mcbin[yvar]     ,100,-2.5,2.5,ax,'green')
        Profile(databin['ScEta'], databin[yvar]   ,100,-2.5,2.5,ax, 'red')
        Profile(mcbin['ScEta']  , mcbin[yvarcorr],100,-2.5,2.5,ax, 'blue')
        fig.savefig(binFileNames[i]+'_'+yvar+"eta.pdf")

        fig = plt.figure()
        ax = fig.add_subplot(111)
        ax.set_title(plotTitles[i])
        ax.set_xlabel('Pt')
        ax.set_ylabel(yvar)
        Profile(mcbin['Pt']  , mcbin[yvar]     ,100,20.,80.,ax, 'green')
        Profile(databin['Pt'], databin[yvar]   ,100,20.,80.,ax, 'red')
        Profile(mcbin['Pt']  , mcbin[yvarcorr],100,20.,80.,ax, 'blue')
        fig.savefig(binFileNames[i]+'_'+yvar+"pt.pdf")

        fig = plt.figure()
        ax = fig.add_subplot(111)
        ax.set_title(plotTitles[i])
        ax.set_xlabel('rho')
        ax.set_ylabel(yvar)
        Profile(mcbin['rho']  , mcbin[yvar]     ,100,0.,40.,ax, 'green')
        Profile(databin['rho'], databin[yvar]   ,100,0.,40.,ax, 'red')
        Profile(mcbin['rho']  , mcbin[yvarcorr],100,0.,40.,ax, 'blue')
        fig.savefig(binFileNames[i]+'_'+yvar+"rho.pdf")

        fig = plt.figure()
        ax = fig.add_subplot(111)
        ax.set_title(plotTitles[i])
        ax.set_xlabel('nvtx')
        ax.set_ylabel(yvar)
        Profile(mcbin['nvtx']  , mcbin[yvar]     ,41,0.,40.,ax, 'green')
        Profile(databin['nvtx'], databin[yvar]   ,41,0.,40.,ax, 'red')
        Profile(mcbin['nvtx']  , mcbin[yvarcorr],41,0.,40.,ax, 'blue')
        fig.savefig(binFileNames[i]+'_'+yvar+"nvtx.pdf")

        fig = plt.figure()
        ax = fig.add_subplot(111)
        ax.set_title(plotTitles[i])
        ax.set_xlabel('Phi')
        ax.set_ylabel(yvar)
        Profile(mcbin['Phi']  , mcbin[yvar]     ,50,-3.14,3.14,ax, 'green')
        Profile(databin['Phi'], databin[yvar]   ,50,-3.14,3.14,ax, 'red')
        Profile(mcbin['Phi']  , mcbin[yvarcorr],50,-3.14,3.14,ax, 'blue')
        fig.savefig(binFileNames[i]+'_'+yvar+"phi.pdf")

dfmc.query('(-1.444 < ScEta & ScEta < 1.444) & (Pt <37) & (rho <12)')
dfdata.query('(-1.444 < ScEta & ScEta < 1.444) & (Pt <37) & (rho <12)')
dfmc.query('(-1.444 < ScEta & ScEta < 1.444) & (Pt <37) & ( rho >=12)')
dfdata.query('(-1.444 < ScEta & ScEta < 1.444) & (Pt <37) & ( rho >=12)')
dfmc.query('(-1.444 < ScEta & ScEta < 1.444) & ( Pt >=37) & (rho <12)')
dfdata.query('(-1.444 < ScEta & ScEta < 1.444) & ( Pt >=37) & (rho <12)')
dfmc.query('(-1.444 < ScEta & ScEta < 1.444) & ( Pt >=37) & ( rho >=12)')
dfdata.query('(-1.444 < ScEta & ScEta < 1.444) & ( Pt >=37) & ( rho >=12)')
dfmc.query('(ScEta < -1.57 | ScEta > 1.57) & (Pt <37) & (rho <12)')
dfdata.query('(ScEta < -1.57 | ScEta > 1.57) & (Pt <37) & (rho <12)')
dfmc.query('(ScEta < -1.57 | ScEta > 1.57) & (Pt <37) & ( rho >=12)')
dfdata.query('(ScEta < -1.57 | ScEta > 1.57) & (Pt <37) & ( rho >=12)')
dfmc.query('(ScEta < -1.57 | ScEta > 1.57) & ( Pt >=37) & (rho <12)')
dfdata.query('(ScEta < -1.57 | ScEta > 1.57) & ( Pt >=37) & (rho <

/Applications/anaconda/lib/python2.7/site-packages/matplotlib/pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)



dfdata.query('(ScEta < -1.57 | ScEta > 1.57) & ( Pt >=37) & ( rho >=12)')
